# Set sharepoint permissions for a specific team/site

This notebook is part of this Notion instruction page [on setting up API to communicate with Sharepoint](https://www.notion.so/Set-app-Azure-App-to-communicate-with-sharepoint-1d38b68cc7b4803d8a61ee7c6e440d2e)

Change the variables in the first cell to match your use case.

In [7]:
import requests

In [ ]:
TENANT_ID = "your-tenant-id"
CLIENT_ID = "your-app-client-id"
CLIENT_SECRET = "your-app-client-secret"
SHAREPOINT_GRAPH_URL = "sharepoint-graph-url"
# "https://graph.microsoft.com/v1.0/sites/<domain>/teams/<teamname>"

In [8]:
GRAPH_SCOPE = "https://graph.microsoft.com/.default"

def get_graph_token():
    url = f"https://login.microsoftonline.com/{TENANT_ID}/oauth2/v2.0/token"
    data = {
        "grant_type": "client_credentials",
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "scope": GRAPH_SCOPE
    }
    response = requests.post(url, data=data)
    response.raise_for_status()
    return response.json()["access_token"]


In [9]:
def get_site_id(token):
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.get(SHAREPOINT_GRAPH_URL, headers=headers)
    response.raise_for_status()
    return response.json()["id"]

In [10]:
def grant_site_permission(site_id, app_client_id, token, role="read"):
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/permissions"
    body = {
        "roles": [role],
        "grantedToIdentities": [
            {
                "application": {
                    "id": app_client_id,
                }
            }
        ]
    }
    response = requests.post(url, headers=headers, json=body)
    response.raise_for_status()
    print("Access granted successfully.")


In [ ]:
if __name__ == "__main__":
    token = get_graph_token()
    print(token)
    site_id = get_site_id(token)
    print("Site ID:", site_id)
    # Grant access to that site
    grant_site_permission(site_id, CLIENT_ID, token, role="read")